## Experimeto SIGPid

In [ ]:
!pip install apyori
!pip install mlxtend  

In [1]:
import pandas as pd
import numpy  as np
import sklearn
import timeit
import os, sys, stat
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix,plot_confusion_matrix, plot_roc_curve
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from apyori import apriori

**Cria a extrutura de diretórios do experimento**

In [4]:
cwd = os.getcwd()
print(cwd)
#Resultados das métricas e tempos de execução
directory = "SVM_Resultados"
parent_dir = "/MLDP/PIS_PRNR/"
parent_dir1 = "/MLDP/PIS_PRNR/"
parent_dir2 = "/MLDP/PIS_SPR/"
parent_dir3 = "/MLDP/PMAR/"
path = os.path.join(parent_dir, directory)
if os.path.exists(cwd+path):
    print(path + ' : Existentes')
else:
    os.makedirs(cwd+path)
    print("Diretório'%s' criado" %directory)

path1 = os.path.join(parent_dir1, directory)
if os.path.exists(cwd+path1):
    print(path1 + ' : Existentes')
else:
    os.makedirs(cwd+path1)
    print("Diretório'%s' criado" %directory)

path2 = os.path.join(parent_dir2, directory)
if os.path.exists(cwd+path2):
    print(path2 + ' : Existentes')
else:
    os.makedirs(cwd+path2)
    print("Diretório'%s' criado" %directory)

path3 = os.path.join(parent_dir3, directory)
if os.path.exists(cwd+path3):
    print(path3 + ' : Existentes')
else:
    os.makedirs(cwd+path3)
    print("Diretório'%s' criado" %directory) 



C:\Users\joner\OneDrive\Documentos\ProjetoMotorola\SigPID\Experimento
/MLDP/PIS_PRNR/SVM_Resultados : Existentes
/MLDP/PIS_PRNR/SVM_Resultados : Existentes
/MLDP/PIS_SPR/SVM_Resultados : Existentes
/MLDP/PMAR/SVM_Resultados : Existentes


## Carrega o subset com 113 permissões oriundas do Drebin_215

In [5]:
permissions_table = pd.read_csv('datasets/DrebinDatasetPermissoes.csv', encoding = 'utf8')


**A abordagem SigPID opera em duas matrizes,
M e B. M representa uma lista de permissões usadas por amostras de
malware e B representa uma lista de permissões usadas por
aplicativos benignos. Sendo que 1 representa o conjuto dos Malwares e 0 o conjunto dos Benignos**

In [6]:

values=[1]
M = permissions_table[permissions_table['class'].isin(values)]


In [7]:
values=[0]
B = permissions_table[permissions_table['class'].isin(values)]


## PRNR

**Para equilibrar as duas matrizes, os autores utilizaram a Equação abaixo
para calcular o suporte de cada permissão no conjunto
de dados maior e, em seguida, dimencionar proporcionalmente o
suporte correspondente para corresponder ao do conjunto de dados menor.**

$S_{b}(P_{j})=\frac{\sum_{i}{Bij}}{Size(_{Bj})}*Size(_{Mj})$

In [8]:
#𝑆𝑏(𝑃𝑗)
def S_B(j):
    sigmaBij = B.sum(axis = 0, skipna = True)[j]
    sizeBj = B.shape[0]
    sizeMj = M.shape[0]
    return (sigmaBij/sizeBj)*sizeMj



**Pj onde j representa as permissão, e Sb (Pj) representa o suporte de j a permissão na matriz B. PRNR pode então
ser implementado usando a Equação abaixo.**
Na fórmula abaixo, R (Pj) representa a taxa de j
a permissão. O resultado de R (Pj) tem um valor que varia entre
[-1, 1]. 
* Se R (Pj) = 1, isso significa que a permissão Pj é apenas
  usado em um conjunto de dados malicioso, que é uma permissão de alto risco.
* Se R (Pj) = -1, isso significa que a permissão Pj só é usada em
  conjunto de dados benigno, que é uma permissão de baixo risco.
* Se R (Pj) = 0,isso significa que Pj tem muito pouco impacto na detecção de malware.

$ R_(P_{j})=\frac{\sum_{i}{Mij}-S_{b}(P_{j})}{\sum_{i}{Mij}+S_{b}(P_{j})}$

In [9]:
def PRNR(j):
    sigmaMij = M.sum(axis = 0, skipna = True)[j]
    S_Bj = S_B(j)
    #print(str(j)+","+str((sigmaMij-S_Bj)/(sigmaMij+S_Bj)))
    return (sigmaMij-S_Bj)/(sigmaMij+S_Bj)

## Calcula o rank da lista de Benignos e salva em TXT 

In [10]:
import  csv
# variavel que armazena uma lista de permissões
# melhorar a geração do arquivo para remover sujeiras como (" e outros)
permissoes =  []
perm = B.drop(columns=['class','android.permission.INTERNET'])

for per in perm:
    permissions_PRNR_ranking = PRNR(per)
    if permissions_PRNR_ranking !=0:
        permissoes.append(per+','+str(permissions_PRNR_ranking))
        #print(per+','+str(permissions_PRNR_ranking))
        per_B= np.concatenate((permissoes,'class'), axis=None)
        with open("datasets/Lista_B_PRNR.txt","w") as f:
            wr = csv.writer(f,delimiter="\n")
            wr.writerow(list(per_B))


## Calcula o suporte da lista de Malwares e salva em TXT

In [11]:
import  csv
# variavel que armazena uma lista de permissões
permissoes =  []
perm = M.drop(columns=['class','android.permission.INTERNET'])
for per in perm:
    permissions_PRNR_ranking = PRNR(per)
    if permissions_PRNR_ranking !=0:  
        permissoes.append(per+','+str(permissions_PRNR_ranking))
        #print(per+','+str(permissions_PRNR_ranking))
        per_M= np.concatenate((permissoes,'class'), axis=None)
        with open("datasets/Lista_M_PRNR.txt","w") as f:
            wr = csv.writer(f,delimiter="\n")
            wr.writerow(list(per_M))

         

## Aplica PIS sobre o PRNR
**Anteriormente criamos 2 listas(benignos e malwares) com seu devido suportes.
Agora carregamos estas listas e colocamos benignos em ordem crecente e malwares em ordem decrecente**
**Então, escolhemos as três principais permissões em ambas as listas para construir
sistema de detecção de malware. Repetimos o processo novamente. A cada incremento é gerado um subset e um log das permissões selecionadas**

In [12]:
# Carrega a lista de suporte gerada anteriormente e ordena os malwares em ordem decrecente
#criar um contador para ir incrementando a cada execução(6,12,18...)
colnames = ['Permissão','rank']
malwares = pd.read_csv('datasets/Lista_M_PRNR.txt', sep = "," , names = colnames)
malwares["rank"] = pd.to_numeric(malwares["rank"])
malwaresort = malwares.sort_values(by=['rank'],ascending=[False])
#aqui defini o incremento ex(head(3) para os 3 primeiros)
#malware_increment = malwaresort['Permissão'].head(3).values

# Carrega a lista de suporte gerada anteriormente e ordena os benignos em ordem crecente
benignos = df_PRNR_txt = pd.read_csv('datasets/Lista_B_PRNR.txt', sep="," , names = colnames)
benignos["rank"] = pd.to_numeric(benignos["rank"])
benignosort = benignos.sort_values(by=['rank'],ascending=[True])
#aqui defini o incremento ex(head(3) para os 3 primeiros
#benigno_increment= benignosort['Permissão'].head(3).values

#usando while para gerar o incremento com top 3 valores de cada lista
counter=3
while counter < len(permissoes)/2+2:
    malware_increment = malwaresort['Permissão'].head(counter).values
    benigno_increment= benignosort['Permissão'].head(counter).values
    # aqui eu crio o arquivo que possui as permissões selecionadas de cada lista 
    PIS =  np.concatenate((benigno_increment, malware_increment,'class'), axis=None)
    with open("MLDP/PIS_PRNR/PIS_PRNR"+str(counter*2)+".txt","w") as f:
                wr = csv.writer(f,delimiter=",")
                wr.writerow(PIS)
    
                
    #Aqui é carregado as permissões geradas no passo anterior e cria os subsets de cada incremento
    # esses subsets são derivados da baseline que possui 113 permissões
    df_permi_drebim = pd.read_csv('datasets/DrebinDatasetPermissoes.csv')
    df_PRNR_txt = pd.read_csv('MLDP/PIS_PRNR/PIS_PRNR'+str(counter*2)+'.txt')
    df_filtrado2 = df_permi_drebim.reindex(columns=[*df_PRNR_txt])
    df_PRNR=df_filtrado2.dropna(axis=1)
    #Subset em csv 
    df_PRNR.to_csv("MLDP/PIS_PRNR/PIS_PRNR"+str(counter*2)+".csv", index=False)

    df_PRNR.isna().values.any()
    print('Nº de permissões ',len(df_PRNR.columns)-1)
    
    counter = counter +3



Nº de permissões  6
Nº de permissões  12
Nº de permissões  18
Nº de permissões  24
Nº de permissões  30
Nº de permissões  36
Nº de permissões  42
Nº de permissões  48
Nº de permissões  54
Nº de permissões  60
Nº de permissões  66
Nº de permissões  72
Nº de permissões  78
Nº de permissões  84
Nº de permissões  90
Nº de permissões  96
Nº de permissões  102
Nº de permissões  108
Nº de permissões  112


## PIS = Aplicar SVM com as top 3 de cada lista e vai incrementando até testar todos os subsets gerados

In [13]:
def SVM(counter):
    with open('MLDP/PIS_PRNR/SVM_Resultados/tempos_PIS_PRNR.txt', 'a') as arquivo:
    #agora basta ir mudando o nome do arquivo para o gerado nos passos anteriores
        dataset_df = pd.read_csv('MLDP/PIS_PRNR/PIS_PRNR'+str(counter)+'.csv', encoding = 'utf8')
        start_time = timeit.default_timer()

        state = np.random.randint(100)
        Y = dataset_df['class']
        X = dataset_df.drop(['class'], axis=1)
        # dividir entre conjuntos de treino e teste
        #note que "test_size" é o tamanho da amostra de teste
        X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.3,random_state=1)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

        from sklearn.svm import SVC
        svm = SVC(kernel='linear', C=1.0, random_state=1)
        svm.fit(X_train,y_train)

        # rótulos de predição para X_test
        y_pred=svm.predict(X_test)

        #Metricas
        """
        tn = Verdadeiro Negativo uma previsão correta prevista como benigno

        fp = Falso Positivo um falso alarme um benigno previsto como malicioso

        tp = Verdadeiro Positivo uma previsão correta (malicioso)

        fn = Falso Negativo um rótulo malicioso previsto como benigno

        """

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        elapsed = timeit.default_timer() - start_time
        from sklearn import metrics 
        print('SVM_tempo',elapsed, file=arquivo)
        # A cada execução salva os resultados em um arquivo chamado metricasSVM.txt, mudar conforme necessidade
        with open('MLDP/PIS_PRNR/SVM_Resultados/metricas_PIS_PRNR.txt', 'a') as arquivo1:
            #ideal é automatizar um contador para cada incremento
            print('PIS_PRNR'+str(counter), file=arquivo1)
            # Precisão do modelo: com que frequência o classificador está correto?
            print("Acurácia:",metrics.accuracy_score(y_test, y_pred), file=arquivo1)
            # Precisão do modelo: qual porcentagem de rótulos positivos está correta?
            print("Precisão:",metrics.precision_score(y_test, y_pred), file=arquivo1)
            # Model Recall: qual porcentagem de tuplas positivas são rotuladas como tal?
            print("Recall:",metrics.recall_score(y_test, y_pred), file=arquivo1)
            # 
            print("F1_Score:",metrics.f1_score(y_test, y_pred), file=arquivo1)
            # FPR: porcentagem de falso positivo de todos os negativos nos dados
            print("FPR:",fp/(fp+tn), file=arquivo1)

In [14]:
#Talvez melhorar esse incremento para algo mais elegante :)
counter=6
while counter <= len(permissoes)+2:
    SVM(counter)
    counter = counter +6

## Plot PIS+PRNR
**O decaimento da acurácia ocoreu após 108 permissões, então 108 são as permissões selecionadas como podemos ver no gráfico abaixo**

![](MLDP/PIS_PRNR/SVM_Resultados/DesempenhoPRNR.png)

In [15]:
#Automatizar o grafico para pegar o Max valor
#df = pd.read_csv('MLDP/PIS_PRNR/SVM_Resultados/metricas_PIS_PRNR1.txt', sep=" ",header=None)
#print(df)

## analiar os dados e carregar o subset com as permissões selecionadas no passo PRNR

In [16]:
#carregar o subset gerado no passo anterior
df_PRNR = pd.read_csv("MLDP/PIS_PRNR/PIS_PRNR108.csv", encoding = 'utf8')
df_PRNR.shape[1]-1

108

In [17]:
df_PRNR1 = df_PRNR.drop(columns=['class'])


In [18]:
def SVM(subset,counter):
    with open('MLDP/PIS_SPR/SVM_Resultados/tempos_PIS_SPR.txt', 'a') as arquivo:

        start_time = timeit.default_timer()

        state = np.random.randint(100)
        Y = subset['class']
        X = subset.drop(['class'], axis=1)
        # dividir entre conjuntos de treino e teste
        #note que "test_size" é o tamanho da amostra de teste
        X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.3,random_state=1)
        #usar k-floud
        #n_estimators=100,max_depth=50
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

        from sklearn.svm import SVC
        svm = SVC(kernel='linear', C=1.0, random_state=1)
        svm.fit(X_train,y_train)

        # rótulos de predição para X_test
        y_pred=svm.predict(X_test)

        #Metricas
        """
        tn = Verdadeiro Negativo uma previsão correta prevista como benigno

        fp = Falso Positivo um falso alarme um benigno previsto como malicioso

        tp = Verdadeiro Positivo uma previsão correta (malicioso)

        fn = Falso Negativo um rótulo malicioso previsto como benigno

        """

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        elapsed = timeit.default_timer() - start_time
       
        print('SVM_SPR',elapsed, file=arquivo)
        with open('MLDP/PIS_SPR/SVM_Resultados/metricas_PIS_SPR.txt', 'a') as arquivo1:
            from sklearn import metrics 
            #Salva os resultados em um arquivo txt.
            print('PIS incremento'+str(counter), file=arquivo1)
            # Precisão do modelo: com que frequência o classificador está correto?
            print("Acurácia:",metrics.accuracy_score(y_test, y_pred), file=arquivo1)
            # Precisão do modelo: qual porcentagem de rótulos positivos está correta?
            print("Precisão:",metrics.precision_score(y_test, y_pred), file=arquivo1)
            # Model Recall: qual porcentagem de tuplas positivas são rotuladas como tal?
            print("Recall:",metrics.recall_score(y_test, y_pred), file=arquivo1)
            # 
            print("F1_Score:",metrics.f1_score(y_test, y_pred), file=arquivo1)
            # FPR: porcentagem de falso positivo de todos os negativos nos dados
            print("FPR:",fp/(fp+tn), file=arquivo1)

## PRNR + SPR
**Classificação de permissão baseada em suporte (SPR): se o suporte de uma permissão for muito
baixo, não terá muito impacto no desempenho da detecção de malware.**

In [19]:
len(df_PRNR1.columns)

108

#### cria um arquivo para cada incremento, fazer isso de 5 em 5

In [20]:
#calcula a frequencia que cada permissão possui
soma = df_PRNR1.sum(axis=0)
frequencia = soma.sort_values(ascending=False)
#melhorar para quando o valor for impar ou par
incremento=5
while incremento < 115:
    freq=frequencia.head(incremento)
    with open("MLDP/PIS_SPR/PIS_SPR"+str(incremento)+".txt","w") as f:
        concatenar =  np.concatenate((freq.index,'class'), axis=None)
        wr = csv.writer(f,delimiter=",")
        wr.writerow(concatenar)
        
    df_PRNR_txt = pd.read_csv('MLDP/PIS_SPR/PIS_SPR'+str(incremento)+'.txt')
    df_filtrado2 = df_permi_drebim.reindex(columns=[*df_PRNR_txt])
    df_PRNR=df_filtrado2.dropna(axis=1)
    #cria o subset que sera testado usando svm
    df_PRNR.to_csv("MLDP/PIS_SPR/PIS_SPR"+str(incremento)+".csv", index=False)

    df_PRNR.isna().values.any()
    print('Nº Permissões: ',len(df_PRNR.columns)-1)  
    incremento = incremento +5


Nº Permissões:  5
Nº Permissões:  10
Nº Permissões:  15
Nº Permissões:  20
Nº Permissões:  25
Nº Permissões:  30
Nº Permissões:  35
Nº Permissões:  40
Nº Permissões:  45
Nº Permissões:  50
Nº Permissões:  55
Nº Permissões:  60
Nº Permissões:  65
Nº Permissões:  70
Nº Permissões:  75
Nº Permissões:  80
Nº Permissões:  85
Nº Permissões:  90
Nº Permissões:  95
Nº Permissões:  100
Nº Permissões:  105
Nº Permissões:  108


In [21]:
counter=5
while counter < 115:
    df_SPR = pd.read_csv('MLDP/PIS_SPR/PIS_SPR'+str(counter)+'.csv', encoding = 'utf8')
    SVM(df_SPR,counter)
    counter = counter +5         

![](MLDP/PIS_SPR/SVM_Resultados/DesempenhoPRNR_SPR.png)

## PRNR + SPR + PMAR
**aqui retiramos as permissões que possuem associação (não retiramos as duas, deixamos apenas 1 delas):**

In [22]:
#Aqui carregamos as permissões extraidas no passo anterior
PRNR_SPR = pd.read_csv("MLDP/PIS_SPR/PIS_SPR30.csv", encoding = 'utf8')
# -1 para não considerar a coluna class
len(PRNR_SPR.columns)-1

30

## PMAR
**se os eventos A e B sempre co-ocorrem, é altamente provável que
esses dois eventos estejam associados. então excluimos a permissão com menor relevancia**
**Por exemplo, a permissão WRITE SMS e a permissão
READ SMS são sempre usadas juntas. Ambos também pertencem à lista
de permissões “perigosas” do Google. Ainda assim, não é necessário
considerar ambas as permissões, pois uma delas é suficiente para
caracterizar determinados comportamentos. Como resultado,
podemos associar um, que tem um suporte maior, ao seu parceiro.Neste exemplo, podemos remover a permissão WRITE SMS**

In [23]:
import sys
import argparse
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules, fpmax, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
#import matplotlib.pyplot as plt


## Read data from the csv file on local system.
data_frame = PRNR_SPR.copy() 

features_name = data_frame.columns.values.tolist()

class_apk = data_frame['class']

features_dataset = data_frame.drop(['class'], axis=1)

num_apk = features_dataset.shape[0] - 1
num_features = features_dataset.shape[1]

records = []
for i in range(0,num_apk):
    if class_apk[i] in [0, 1]:
        i_list = []
        for j in range(0,num_features):
            if features_dataset.values[i][j] == 1:
                i_list.append(features_name[j])
        records.append(i_list) 
#print(records)

te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)

#freq_items = apriori(df, min_support=0.1, use_colnames=True, low_memory=True, max_len =5, verbose=1)
#freq_items = fpgrowth(df, min_support=0.2, use_colnames=True, max_len =3, verbose=1)
freq_items = apriori(df, 
                    min_support=0.1,
                    use_colnames=True,
                    max_len =2,
                    verbose=1)

#print(freq_items)

rules = association_rules(freq_items, metric="confidence", min_threshold=0.965)
#rules = association_rules(freq_items)

rules = rules[(rules['lift'] >= 0.0)]
rules_list = list(rules)
print(rules_list[0])
print(rules[['antecedents', 'consequents', 'support','confidence','lift']])

Processing 552 combinations | Sampling itemset size 2
antecedents
                              antecedents  \
0  (android.permission.CHANGE_WIFI_STATE)   
1    (android.permission.MANAGE_ACCOUNTS)   
2          (android.permission.WRITE_SMS)   

                              consequents   support  confidence      lift  
0  (android.permission.ACCESS_WIFI_STATE)  0.160758    0.993016  2.285669  
1       (android.permission.GET_ACCOUNTS)  0.103359    0.992971  3.323536  
2           (android.permission.READ_SMS)  0.111407    0.984136  5.269405  


In [24]:
PRNR_SPR = pd.read_csv("MLDP/PIS_SPR/PIS_SPR30.csv", encoding = 'utf8')
PRNR_SPR_PMAR = PRNR_SPR.drop(columns=['android.permission.WRITE_SMS','android.permission.ACCESS_WIFI_STATE','android.permission.MANAGE_ACCOUNTS'])
#subsetfinal
PRNR_SPR_PMAR.to_csv("MLDP/PMAR/PRNR_SPR_PMAR.csv", index=False)
print('Nº de Permissões',len(PRNR_SPR_PMAR.columns)-1)

Nº de Permissões 27


In [25]:
PRNR_SPR_PMAR.columns.values

array(['android.permission.ACCESS_NETWORK_STATE',
       'android.permission.WRITE_EXTERNAL_STORAGE',
       'android.permission.READ_PHONE_STATE',
       'android.permission.WAKE_LOCK',
       'android.permission.RECEIVE_BOOT_COMPLETED',
       'android.permission.VIBRATE', 'android.permission.GET_ACCOUNTS',
       'android.permission.ACCESS_FINE_LOCATION',
       'android.permission.ACCESS_COARSE_LOCATION',
       'android.permission.SEND_SMS', 'android.permission.READ_CONTACTS',
       'android.permission.RECEIVE_SMS', 'android.permission.READ_SMS',
       'android.permission.GET_TASKS',
       'android.permission.CHANGE_WIFI_STATE',
       'android.permission.WRITE_SETTINGS', 'android.permission.CAMERA',
       'android.permission.CALL_PHONE',
       'android.permission.WRITE_CONTACTS',
       'android.permission.READ_EXTERNAL_STORAGE',
       'android.permission.USE_CREDENTIALS',
       'com.android.browser.permission.READ_HISTORY_BOOKMARKS',
       'android.permission.CHANGE_NETW